In [1]:
import os
import torch
import json
import argparse
from tqdm import tqdm
import torch.optim as optim
from torchvision.utils import save_image

from training_models import MyClip

from datasets import collate_fn_image, ImageDataset

from torchvision import transforms

In [2]:
dataset_name = "mini_MathVista_grid"
args = {
    "model": "clip",
    "poison_save_pth": f"./data/poisons/{dataset_name}",
    "iter_attack": 500,
    "lr_attack": 0.2,
    "base_data_pth": f"data/{dataset_name}/base",
    "target_data_pth": f"data/{dataset_name}/target",
    "questions_pth": f"data/{dataset_name}/questions.json",
    "temperature": 0,
    "max_new_tokens": 200,
    "eps": 16,
}
def read_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

encoder_model = MyClip()

question_json = read_json(args["questions_pth"])
pids = question_json.keys()



base_caps = read_json(os.path.join(args["base_data_pth"], "caps.json"))
target_caps = read_json(os.path.join(args["target_data_pth"], "caps.json"))

In [47]:
def collate_fn_image(batch):
    image, caps = zip(*batch)
    image = torch.stack(image, dim=0)
    bsz, c, h, w = image.shape
    assert bsz == 1
    
    return image, caps[0]

In [48]:
base_dataset = ImageDataset(base_caps["annotations"])
target_dataset = ImageDataset(target_caps["annotations"])

base_dataloader = torch.utils.data.DataLoader(
    base_dataset, batch_size=1, 
    shuffle=False, collate_fn=collate_fn_image
)

target_dataloader = torch.utils.data.DataLoader(
    target_dataset, batch_size=1, 
    shuffle=False, collate_fn=collate_fn_image
)

torch.Tensor